In [52]:
import sys
sys.path.append("..")
import imagenet_load
from importlib import reload
reload(imagenet_load)
import tarfile
import numpy as np
import imread
%pylab inline
import imread
from imread import imread_from_blob
from imagenet_load import orient
import multigpu
reload(multigpu)
import filter_gen
import conv
reload(conv)
import gc
import logging
import dill
from scipy import linalg
from sklearn import metrics

Populating the interactive namespace from numpy and matplotlib


/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['linalg', 'imread']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [66]:
NUM_CLASSES = 100

In [65]:
%time loader = imagenet_load.ImagenetLoader(NUM_CLASSES, classes_path="../classes", mmap_loc="/dev/shm/imagenet_train", n_procs=16)

NameError: name 'NUM_CLASSES' is not defined

In [55]:
%time X = loader.load_all()

Loaded 0 classes into mem
CPU times: user 64 ms, sys: 328 ms, total: 392 ms
Wall time: 18 s


In [57]:
def conv_mmap(*args,
         **kwargs):
         from importlib import reload
         import conv
         import multigpu
         reload(conv)
         reload(multigpu)
         sidx = kwargs["sidx"]
         eidx = kwargs["eidx"]
         mmap_in_loc = kwargs["mmap_in_loc"]
         mmap_in_shape = kwargs["mmap_in_shape"]
         X = np.memmap(mmap_in_loc, mode="r", dtype='uint8', shape=mmap_in_shape)
         mmap_out_shape = conv.conv_compute_output_shape(data=X, **kwargs)
         X = X[sidx:eidx,:,:,:]
         mmap_out_loc = kwargs["mmap_out_loc"]   
         X_out = np.memmap(mmap_out_loc, mode="r+", dtype='float32', shape=mmap_out_shape)[sidx:eidx,:,:,:]
         X_out_local, filters = conv._conv(X,
                      filter_gen=kwargs["filter_gen"],                                                                                                                                                                         
                      num_feature_batches=kwargs["num_feature_batches"],                                    
                      feature_batch_size=kwargs["feature_batch_size"],
                      data_batch_size=kwargs["data_batch_size"],
                      pool_type=kwargs["pool_type"],
                      pool_size=kwargs["pool_size"],
                      pool_stride=kwargs["pool_stride"],
                      pad=kwargs["pad"],
                      bias=kwargs["bias"],
                      patch_size=kwargs["patch_size"],
                      conv_stride=kwargs["conv_stride"])
         np.copyto(X_out, X_out_local)
         X_out.flush()
         return multigpu.MmapResult(mmap_out_loc, 'float32', mmap_out_shape, "r+", range(sidx,eidx))

In [139]:
gpu_handler = multigpu.MultiGpuHandler(16)
gpu_handler.start_and_wait_for_gpu_init()


Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5105)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using gpu device 1: Tesla K80 (CNMeM is disabled, cuDNN 5105)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using gpu device 2: Tesla K80 (CNMeM is disabled, cuDNN 5105)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  

In [143]:
conv_args_template = \
{ 
  "mmap_in_loc":"/dev/shm/imagenet_train",
  "mmap_in_shape":X.shape,
  "mmap_out_loc": "/dev/shm/X_lift",
  "filter_gen":fg,
  "num_feature_batches":8,
  "data_batch_size":128,
  "feature_batch_size":64,
  "pool_size":128,
  "pool_type":"avg",
  "pool_stride":128,
  "patch_size":6,
  "pad":0,
  "bias":1,
  "conv_stride":1
}


def chunk_idxs(size, chunks):
    chunk_size  = int(np.ceil(size/chunks))
    idxs = list(range(0, size+1, chunk_size))
    idxs.append(size)
    return list(zip(idxs[:-1], idxs[1:]))

mmap_out_shape = conv.conv_compute_output_shape(data=X, **conv_args_template)
X_out = np.memmap("/dev/shm/X_lift", mode="w+", dtype="float32", shape=mmap_out_shape)

In [145]:
gamma  = 1e-3
results = {}

In [146]:
mmap_out_shape

(129395, 1024, 2, 2)

In [ ]:
for gamma in [1e-1,1,10,100]:
    fg = filter_gen.make_gaussian_filter_gen(gamma, patch_size=6, seed=0)
    gpu_results = []
    for gpu, (sidx, eidx) in list(zip(gpu_handler.gpus,chunk_idxs(X.shape[0], 16))):
            conv_args = conv_args_template.copy()
            conv_args['sidx'] = sidx
            conv_args['eidx'] = eidx
            gpu_result = gpu.submit_async(conv.conv_mmap, **conv_args)
            gpu_results.append(gpu_result)

    for gpu_result in gpu_results:
        %time gpu_result.result()

    X_train = X_out.reshape(X_out.shape[0], -1)
    reg = 1e-8

    %time XtX = X_train.T.dot(X_train)

    Y = np.eye(NUM_CLASSES).astype('float32')[loader.Y.astype('int')]

    %time XtY = X_train.T.dot(Y)

    %time XtX = XtX.astype('float64')
    %time XtY = XtY.astype('float64')

    idx = np.diag_indices(XtX.shape[0])

    XtX[idx] += reg
    %time model = scipy.linalg.solve(XtX, XtY, sym_pos=True)
    XtX[idx] -= reg

    y_pred = X_train.dot(model)

    labels_pred = np.argmax(y_pred, axis=1)

    results[gamma] = metrics.accuracy_score(labels_pred, loader.Y)
    print(results[gamma])

In [127]:
results

{0.1: 0.13486610765485529,
 1: 0.24374203021755092,
 10: 0.21613663588237567,
 100: 0.016082537965145485}